In [1]:
import pandas as pd
from src.model.generate_stories import tfmclassifier

In [2]:
from transformers import AutoTokenizer, AutoModelWithLMHead
  
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", output_hidden_states=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
C:\Users\edbon\anaconda3\envs\tf\lib\site-packages\transformers\models\auto\modeling_auto.py:660: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [31]:
type(model)

transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel

In [3]:
emb = tfmclassifier(["этот текст"], model, tokenizer, gen_len=1946, device='cpu')

torch.Size([1, 1946, 768])


In [6]:
emb.size()

torch.Size([1, 768])

In [8]:
a = (1, None)
print(list(a[1:]))

[None]


In [10]:
import torch
print(torch.cuda.memory_stats()['allocated_bytes.all.current'] // 1024 // 1024)

KeyError: 'allocated_bytes.all.current'